In [8]:
import requests
import os
from dotenv import load_dotenv
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Visualization imports
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
# Configure matplotlib
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
# Load API key
load_dotenv()
finnhub_key = os.getenv("FINNHUB_API_KEY")
if not finnhub_key:
    raise ValueError("⚠️ Please add your FINNHUB_API_KEY to .env file")
print("✅ All libraries loaded and API key ready")

✅ All libraries loaded and API key ready


In [9]:
def clean_text(text):
    if not text:
        return ""
    text = text.strip()
    text = re.sub(r"[^\w\s.,?!]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment
def get_sentiment_vader(text):
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']
    if compound >= 0.05:
        return "Bullish"
    elif compound <= -0.05:
        return "Bearish"
    else:
        return "Neutral"

def extract_keywords(news_list):
    """Extract top 10 keywords from headlines"""
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
                  'of', 'with', 'is', 'are', 'was', 'were', 'be', 'been', 'has', 'have'}
    all_words = []
    for item in news_list:
        words = re.findall(r'\b[a-z]{4,}\b', item['title'].lower())
        all_words.extend([w for w in words if w not in stop_words])
    return Counter(all_words).most_common(10)

In [10]:
def generate_summary(ticker, news, overall_signal, top_keywords):
    """Generate a readable summary from headlines"""
    total = len(news)
    sentiments = [get_sentiment_vader(item['title']) for item in news]
    bullish = sentiments.count('Bullish')
    bearish = sentiments.count('Bearish')

    # Build keyword phrase
    keyword_phrase = ", ".join([word for word, count in top_keywords[:5]])

    # Build summary based on overall signal
    if overall_signal == "Bullish":
        signal_text = f"Positive momentum detected with {bullish} out of {total} headlines showing bullish sentiment."
    elif overall_signal == "Bearish":
        signal_text = f"Negative pressure detected with {bearish} out of {total} headlines showing bearish sentiment."
    else:
        signal_text = f"No clear direction with {bullish} bullish and {bearish} bearish headlines out of {total} total."

    summary = (
        f"{ticker} Market Summary\n"
        f"{'=' * 50}\n"
        f"Signal: {overall_signal}\n"
        f"{signal_text}\n"
        f"Key Topics: {keyword_phrase}\n"
        f"{'=' * 50}\n"
    )

    return summary


def fetch_earnings(ticker):
    """Fetch earnings data from Finnhub"""
    url = f"https://finnhub.io/api/v1/earnings?symbol={ticker}&token={finnhub_key}"
    try:
        data = requests.get(url).json()
        earnings = data.get("earnings", [])
        if earnings:
            latest = earnings[0]
            return {
                "symbol": latest.get("symbol", "N/A"),
                "actual_eps": latest.get("actual", "N/A"),
                "estimated_eps": latest.get("estimate", "N/A"),
                "surprise": latest.get("surprise", "N/A"),
                "surprise_percent": latest.get("surprisePercent", "N/A"),
                "reported_date": latest.get("reportedDate", "N/A")
            }
        else:
            return None
    except Exception as e:
        print(f"Error fetching earnings: {e}")
        return None

print("✅ Summary and Earnings functions ready")


✅ Summary and Earnings functions ready


In [11]:
def fetch_news(ticker, from_date="2026-01-10", to_date="2026-01-14"):
    url = f"https://finnhub.io/api/v1/company-news?symbol={ticker}&from={from_date}&to={to_date}&token={finnhub_key}"
    try:
        data = requests.get(url).json()
    except Exception as e:
        print("Error fetching news:", e)
        return []
    headlines = []
    for item in data[:10]:
        title = clean_text(item.get("headline", ""))
        link = item.get("url", "")
        headlines.append({"title": title, "link": link})
    if not headlines:
        print("⚠️ No headlines found. Check ticker or date range!")
    return headlines


In [12]:
ticker = input("Enter stock ticker (e.g., AAPL): ").upper().strip()
from_date = input("Enter start date (YYYY-MM-DD): ").strip()
to_date = input("Enter end date (YYYY-MM-DD): ").strip()

# Fetch news dynamically based on user input
news = fetch_news(ticker, from_date, to_date)
if not news:
    print("❌ No news found for this ticker/date range!")
else:
    print(f"✅ Collected {len(news)} headlines for {ticker}")
    for i, item in enumerate(news, 1):
        title = item['title']
        sentiment = get_sentiment_vader(title)
        print(f"{i}. {title}")
        print(f"   Link: {item['link']}")
        print(f"   Sentiment: {sentiment}\n")
    print(f"✅ Collected {len(news)} headlines for visualization")

⚠️ No headlines found. Check ticker or date range!
❌ No news found for this ticker/date range!


In [13]:
if news:
    sentiments = [get_sentiment_vader(item['title']) for item in news]
    sentiment_counts = {
        'Bullish': sentiments.count('Bullish'),
        'Bearish': sentiments.count('Bearish'),
        'Neutral': sentiments.count('Neutral')
    }
    plt.figure(figsize=(10, 7))
    colors = ['#06D6A0', '#EF476F', '#FFD166']
    explode = (0.05, 0.05, 0.05)
    wedges, texts, autotexts = plt.pie(
        sentiment_counts.values(),
        labels=sentiment_counts.keys(),
        autopct=lambda p: f"{p:.1f}%\n({int(p*sum(sentiment_counts.values())/100)})",
        colors=colors,
        explode=explode,
        startangle=90,
        textprops={'fontsize': 12, 'fontweight': 'bold'}
    )
    for text in texts + autotexts:
        text.set_color('black')
        text.set_fontsize(12)
        text.set_fontweight('bold')
    plt.title(f'Sentiment Distribution for {ticker}', fontsize=16, fontweight='bold', pad=20)
    filename = f"{ticker}_sentiment_pie.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"📊 Saved: {filename}")
    plt.show()
else:
    print("❌ No data to visualize")


❌ No data to visualize


In [14]:
if news:
    # Sentiment counts
    sentiments = [get_sentiment_vader(item['title']) for item in news]
    bullish = sentiments.count('Bullish')
    bearish = sentiments.count('Bearish')
    neutral = sentiments.count('Neutral')
    total = len(news)

    # Overall signal
    if bullish > bearish:
        overall_signal = "Bullish"
    elif bearish > bullish:
        overall_signal = "Bearish"
    else:
        overall_signal = "Neutral"

    # Top keywords
    top_keywords = extract_keywords(news)

    # Fetch earnings
    earnings = fetch_earnings(ticker)

    # Generate summary
    summary = generate_summary(ticker, news, overall_signal, top_keywords)
    print(summary)

    # Print earnings if available
    if earnings:
        print(f"\n📊 Latest Earnings Report for {ticker}")
        print("=" * 50)
        print(f"Reported Date:    {earnings['reported_date']}")
        print(f"Actual EPS:       {earnings['actual_eps']}")
        print(f"Estimated EPS:    {earnings['estimated_eps']}")
        print(f"Surprise:         {earnings['surprise']}")
        print(f"Surprise %:       {earnings['surprise_percent']}%")
        print("=" * 50)
    else:
        print(f"\n⚠️  No earnings data available for {ticker}")

    # Structured final report
    final_report = {
        "ticker": ticker,
        "date_range": f"{from_date} to {to_date}",
        "headlines_analyzed": total,
        "bullish": bullish,
        "bearish": bearish,
        "neutral": neutral,
        "overall_signal": overall_signal,
        "top_keywords": top_keywords,
        "earnings": earnings,
        "summary": summary
    }
    print("\n📄 Structured Report:")
    print(final_report)
else:
    print("❌ No news found for this ticker/date range!")

❌ No news found for this ticker/date range!
